It is recommended to read the explanation in README.md first

In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

Inspired by https://arxiv.org/abs/1702.08591 (looks-linear init), https://arxiv.org/abs/1711.04735 (dynamical isometry), and https://openreview.net/forum?id=Fp7__phQszn (inductive bias of preserving original orientation of features with almost linear input-output mapping)

In [ ]:
class CustomLinearLayer(nn.Module):
    def __init__(self, input_size, output_size, init="looks_linear"):
        super(CustomLinearLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size, bias=True)
        nn.init.zeros_(self.linear.bias)

        if init == "zero":
            nn.init.zeros_(self.linear.weight)
        elif init == "looks_linear":
            if input_size * 2 != output_size:
                raise ValueError("Output size must be twice that of input size")
                
            with torch.no_grad():
                weight = torch.zeros(input_size * 2, input_size)

                for i in range(self.linear.in_features):
                    weight[2 * i, i] = 1
                    weight[2 * i + 1, i] = -1

                self.linear.weight.copy_(weight)
                nn.init.zeros_(self.linear.bias)
                
            """ Example matrix: [
                [1, 0, 0],
                [-1, 0, 0],
                [0, 1, 0],
                [0, -1, 0],
                [0, 0, 1],
                [0, 0, -1]
            ] """

    def forward(self, x):
        return self.linear(x)

General idea of this approach is to start with a simple 1-layer model (no hidden layer or activation function) initialized to 0 weights. Train the model and use the predictions generated by the model as prev_output (see below) for the second model, which means the next model will essentially be training to fit the residuals between the predictions generated by the first model and the actual outcomes. The second model will be a two-layer model, where the first layer is initialized with looks-linear initialization and the second layer initialized to 0 weights, with the activation function after the first layer being ReLU. Train the second model and use the predictions generated by the second model as prev_output for the third model. Since the second model generate an intermediate features, where the number of intermediate features is 2 times the original features, we append the intermediate features to the list of features for use in the third model. Then, train the third model in similar fashing as second model. Rinse and repeat until validation loss of this approach no longer decreases.

Example:
- Train 1st model with 1-layer neural network (no hidden layer, essentially just doing linear regression), with zero weight initialization
- Obtain the predictions generated by 1st model as prev_output for the second model
- Adjust learning rate appropriately and train 2nd model with 2-layer neural network (input and output layer), where activation function is ReLU, with "looks-linear" initliazation for first layer and zero weight initialization for second layer
- Obtain the features generated by the 1st layer of the 2nd model with self.extract_features()
- Scale the features to zero mean and variance of one
- Append the features to the feature list. The feature list will now be 3 times its original size as "looks-linear" initialization generate input_size * 2 amount of features
- Train 3rd model in the same manner as the 2nd model, except the input features will be 3x the input features of 1st and 2nd model
- Rinse and repeat until validation loss no longer decreases

This architecture is independently discovered and is similar, but not the same, as this paper https://arxiv.org/abs/2002.07971. 

In [ ]:
class DenseGrowNet(nn.Module):        
    def __init__(self, input_size, output_size, is_first_model):
        super(DenseGrowNet, self).__init__()
        self.activation = nn.ReLU()
        
        self.input_size = input_size
        self.output_size = output_size
        
        self.is_first_model = is_first_model
                        
        if is_first_model:
            self.first_layer = CustomLinearLayer(input_size, output_size, init="zero")
        else:
            self.first_layer = CustomLinearLayer(input_size, input_size * 2, init="looks_linear")
            self.last_layer = CustomLinearLayer(input_size * 2, output_size, init="zero")

    def forward(self, x, prev_output=None):
        if self.is_first_model and prev_output is not None:
            raise ValueError("This is the first model and prev_output is passed in")
        if not self.is_first_model and prev_output is None:
            raise ValueError("This is not the first model and prev_output is not passed in")
        
        if prev_output is None:
            return self.first_layer(x)
        
        x = self.activation(self.first_layer(x))
        return self.last_layer(x) + prev_output
    
    def extract_features(self, x, prev_output=None):
        if self.is_first_model:
            raise ValueError("This is not intended for the first model")
        if not self.is_first_model and prev_output is None:
            raise ValueError("This is not the first model and prev_output is not passed in")
            
        return self.activation(self.first_layer(x))

Custom Loss below with L1 and L2 regularization. It is only recommended to use with SGD optimizer. There may be issues with adaptive gradient algorithm when doing navive L1/L2 regularization as mentioned in this paper https://arxiv.org/abs/1711.05101 (not sure if AdamW actually works better in practice consensus wise). 

L1 regularization is particularly beneficial as it is rotationally invariant and robust against uninformative features, which are also the inductive biases that contribute to tree-based models strong performance on tabular data https://openreview.net/forum?id=Fp7__phQszn

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, l1_lambda, l2_lambda):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    def forward(self, outputs, labels, model):
        loss = self.criterion(outputs, labels)
        
        l1_norm = sum(p.abs().sum() for name, p in model.named_parameters() if 'bias' not in name)
        l2_norm = sum(p.pow(2.0).sum() for name, p in model.named_parameters() if 'bias' not in name)
        
        loss += self.l1_lambda * l1_norm + self.l2_lambda * l2_norm
        return loss